In [7]:
import numpy as np

import imageio, sys, torch
from PIL import Image
sys.path.append("third_party/d2net/")
from lib.model_test import D2Net
from lib.utils import preprocess_image
from lib.pyramid import process_multiscale
import cv2

def d2net_features(im_path, max_size = 640):
    # CUDA
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    # Creating CNN model
    model = D2Net(
        model_file="/home/niranjan/recons3d/third_party/d2net/models/d2_tf_no_phototourism.pth",
        use_relu=True,
        use_cuda=use_cuda
    )
    image = imageio.imread(im_path)
    if len(image.shape) == 2:
        image = image[:, :, np.newaxis]
        image = np.repeat(image, 3, -1)

    # TODO: switch to PIL.Image due to deprecation of scipy.misc.imresize.
    resized_image = image
    if max(resized_image.shape) > max_size:
        print(max_size / max(resized_image.shape))
        resized_image = cv2.resize(
            resized_image, None, None,
            max_size / max(resized_image.shape), max_size / max(resized_image.shape)
        ).astype('float')
    if sum(resized_image.shape[: 2]) > 2800:
        resized_image = cv2.resize(
            resized_image, None, None,
            2800 / sum(resized_image.shape[: 2]), 2800 / sum(resized_image.shape[: 2])
        ).astype('float')

    fact_i = image.shape[0] / resized_image.shape[0]
    fact_j = image.shape[1] / resized_image.shape[1]

    input_image = preprocess_image(
        resized_image,
        preprocessing="caffe"
    )
    with torch.no_grad():        
        keypoints, scores, descriptors = process_multiscale(
            torch.tensor(
                input_image[np.newaxis, :, :, :].astype(np.float32),
                device=device
            ),
            model,
            scales=[1]
        )

    # Input image coordinates
    keypoints[:, 0] *= fact_i
    keypoints[:, 1] *= fact_j
    # i, j -> u, v
    keypoints = keypoints[:, [1, 0, 2]]
    return {"keypoints":keypoints, "descriptors": descriptors, "scores":scores}
    

In [8]:
res = d2net_features("/home/niranjan/recons3d/dataset/train/church/images/00002.png")

/var/tmp/ipykernel_256724/248916581.py:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(im_path)


0.625


RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)